<a href="https://colab.research.google.com/github/rohanrjp/rental_DVD_regression/blob/main/DVD_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PROBLEM STATEMENT**

A DVD rental company wants to figure out how many days a customer will rent a DVD for based on some features.The company wants a model which yields a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

Import necessary modules and models

*   Lasso:For feature selection
*   Two regression models:

                          1.   Linear Regression
                          2.   Random Forest Regressor







In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

Import Rental DVD data

The data they provided is in the csv file rental_info.csv. It has the following features:

"rental_date": The date (and time) the customer rents the DVD.
"return_date": The date (and time) the customer returns the DVD.
"amount": The amount paid by the customer for renting the DVD.
"amount_2": The square of "amount".
"rental_rate": The rate at which the DVD is rented for.
"rental_rate_2": The square of "rental_rate".
"release_year": The year the movie being rented was released.
"length": Lenght of the movie being rented, in minuites.
"length_2": The square of "length".
"replacement_cost": The amount it will cost the company to replace the DVD.
"special_features": Any special features, for example trailers/deleted scenes that the DVD also has.
"NC-17", "PG", "PG-13", "R": These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [ ]:
df=pd.read_csv("/rental_info.csv")
#print(df.info())
df["return_date"]=pd.to_datetime(df["return_date"])
df["rental_date"]=pd.to_datetime(df["rental_date"])
#print(df.info())
df["rental_length"]=df["return_date"]-df["rental_date"]
df["rental_length_days"]=df["rental_length"].dt.days
df["deleted_scenes"]=np.where(df["special_features"].str.contains("Deleted Scenes"),1,0)
df["behind_the_scenes"]=np.where(df["special_features"].str.contains("Behind the Scenes"),1,0)
df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length,rental_length_days,deleted_scenes,behind_the_scenes
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3 days 20:46:00,3,0,1
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2 days 20:05:00,2,0,1
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7 days 05:44:00,7,0,1
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2 days 02:24:00,2,0,1
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4 days 01:05:00,4,0,1


In [ ]:
cols_to_drop=["special_features","rental_length","rental_length_days","rental_date","return_date"]
X=df.drop(cols_to_drop,axis=1)

Y=df["rental_length_days"]


In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=9)

In [ ]:
lasso=Lasso(alpha=0.2,random_state=9)
lasso.fit(X_train,Y_train)
lasso_coef=lasso.coef_

X_lasso_train,X_lasso_test=X_train.iloc[:,lasso_coef>0],X_test.iloc[:, lasso_coef > 0]

ols=LinearRegression()
ols=ols.fit(X_lasso_train,Y_train)
Y_test_pred=ols.predict(X_lasso_test)
mse_lin_reg_lasso=mean_squared_error(Y_test,Y_test_pred)
print(f"Linear Regression with Lasso provides a MSE of {mse_lin_reg_lasso:.4f}")

ols_nol=LinearRegression()
ols_nol=ols_nol.fit(X_train,Y_train)
y_pred=ols_nol.predict(X_test)
mse_lin_reg=mean_squared_error(Y_test,y_pred)

print(f"Linear Regression without Lasso provides a MSE of {mse_lin_reg:.4f}")

print(f"Thus we can say Lasso is not required for accurate Linear Regression")


Linear Regression with Lasso provies a MSE of 4.8123
Linear Regression without Lasso provies a MSE of 2.9417
Thus we can say Lasso is not required for accurate Linear Regression


In [ ]:
rf=RandomForestRegressor()
param_dist = {'n_estimators': np.arange(1,101,1),'max_depth':np.arange(1,11,1)}
rand_search = RandomizedSearchCV(rf,
                                 param_distributions=param_dist,
                                 cv=5,
                                 random_state=9)
rand_search.fit(X_train, Y_train)
hyper_params = rand_search.best_params_
rf = RandomForestRegressor(n_estimators=hyper_params["n_estimators"],
                           max_depth=hyper_params["max_depth"],
                           random_state=9)
rf.fit(X_train,Y_train)
rf_pred = rf.predict(X_test)
mse_random_forest= mean_squared_error(Y_test, rf_pred)
print(mse_random_forest)
print(f"Random Forest Regression provides a MSE of {mse_random_forest:.4f}")

print(f"Thus we can say Random Forest is the best model for this task and provides a minimum MSE of {mse_random_forest:.4f}")



2.225667528098759
Random Forest Regression provides a MSE of 2.2257
Thus we can say Random Forest is the best model for this task and provides a minimum MSE of 2.2257
